## SimEx - Main Function

In [1]:
# IMPORT LIBRARIES
import numpy as np

from components_configuration import selectedModifier,selectedSimulator,selectedValidator
from global_settings import simexSettings,mds

from ValidatorController import ValidatorController
from ModifierController import ModifierController
from SimulatorController import SimulatorController
from Logger import Logger

validator_controller = ValidatorController()
logger = Logger()
logger_main_arguments = {}
is_main_func = True
# Initialize interval list for the first iteration
intervals_list=[[mds['domain_min_interval'], mds['domain_max_interval']]]


while is_main_func:
    
    # Calls Modifier Controller
    mod_outcome = ModifierController.modifierController(intervals_list=intervals_list,selectedModifier=selectedModifier['selectedModifier'], do_plot=simexSettings['do_plot'])
    mod_x_list = mod_outcome[0]
    checked_intervals = mod_outcome[1]
    print("MAIN mod outcome", mod_outcome)  

    
    # breaks loop if iterations end by granularity reached
    if not mod_x_list: # FALSE IF ['modifier_data_point'] < mdv['modifier_incremental_unit']:
        logger_main_arguments['log_contex'] = 'overall MAIN stats'
        logger_main_arguments['main_status'] = 'no generated points'
        logger_main_arguments['remaining_unfit_intervals'] = checked_intervals
        logger.log_main(logger_main_arguments)
        break
    
    # Calls Simulator
    mod_x,sim_y_list = SimulatorController.simulatorController(mod_x_list,selectedSimulator=selectedSimulator['selectedSimulator'])
    assert len(mod_x) == len(sim_y_list)

    print("MAIN modx", mod_x)  

    # Calls Validator controller
    intervals_list =  validator_controller.validatorController(mod_x_list=np.array(mod_x),sim_y_list=np.array(sim_y_list), selectedValidator=selectedValidator['selectedValidator'],global_interval=[mds["domain_min_interval"], mds["domain_max_interval"]])
    print("MAIN interval list from VAL:", intervals_list)
    
    # Updates interval_list to new range output from validator controller
    # No more unfit intervals -> write MAIN log
    if not intervals_list:
        is_main_func = False
        logger_main_arguments['log_contex'] = 'overall MAIN stats'
        logger_main_arguments['main_status'] = 'no unfit intervals'
        logger.log_main(logger_main_arguments)

# MAIN cycle completed/interrupted -> write OVERALL statistics        
logger_main_arguments['log_contex'] = 'Overall Stats'                          
logger_main_arguments['main_status'] = 'end cycle'
logger.log_main(logger_main_arguments)



Modifier...
[MODC]: intervals list:  [(0, 100)]


ValueError: not enough values to unpack (expected 2, got 1)